In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.semi_supervised import LabelPropagation
from nltk.corpus import stopwords
from xgboost import XGBClassifier
import string
import nltk
import numpy as np

nltk.download('stopwords')

# Carregar o CSV
df = pd.read_csv('./dados/pre-processed.csv')

# Converter os rótulos para valores numéricos
df['label'] = df['label'].apply(lambda x: 1 if x == 'fake' else 0)

# Função para limpar as manchetes (remoção de pontuação e stopwords)
def limpar_texto(texto):
    # Remover pontuações
    texto = ''.join([char for char in texto if char not in string.punctuation])
    # Converter para minúsculas
    texto = texto.lower()
    # Remover stopwords
    stop_words = set(stopwords.words('portuguese'))
    texto = ' '.join([word for word in texto.split() if word not in stop_words])
    return texto

# Aplicar a função de limpeza nas manchetes
df['manchete_limpa'] = df['preprocessed_news'].apply(limpar_texto)

# Inicializar o vetorizador TF-IDF
tfidf = TfidfVectorizer()

# Aplicar o vetorizador nas manchetes limpas
X_tfidf = tfidf.fit_transform(df['manchete_limpa'])

# Separar a variável preditora (X) e a variável resposta (y)
X = X_tfidf  # A matriz TF-IDF é a nossa variável preditora
y = df['label']  # Coluna 'label' como variável resposta

# Dividir o dataset em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Definir diferentes níveis de omissão de rótulos
proporcoes = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# DataFrame para armazenar resultados
resultados = []

for prop in proporcoes:
    # Copiar y_train
    y_train_semi = y_train.copy()
    
    # Determinar quantos rótulos serão removidos
    n_remover = int(prop * len(y_train_semi))
    indices_remover = np.random.choice(y_train_semi.index, n_remover, replace=False)
    
    # Remover (ocultar) esses rótulos configurando-os como -1
    y_train_semi.loc[indices_remover] = -1

    # --- Cenário com LabelPropagation ---
    lp = LabelPropagation(kernel='knn')
    lp.fit(X_train, y_train_semi)
    y_train_propagado = lp.transduction_

    # Treinar Logistic Regression com LabelPropagation
    modelo_log_lp = LogisticRegression(max_iter=1000)
    modelo_log_lp.fit(X_train, y_train_propagado)
    y_pred_log_lp = modelo_log_lp.predict(X_test)
    acc_log_lp = accuracy_score(y_test, y_pred_log_lp)
    report_log_lp = classification_report(y_test, y_pred_log_lp, output_dict=True)

    # Treinar XGBoost com LabelPropagation
    modelo_xgb_lp = XGBClassifier(eval_metric='logloss')
    modelo_xgb_lp.fit(X_train, y_train_propagado)
    y_pred_xgb_lp = modelo_xgb_lp.predict(X_test)
    acc_xgb_lp = accuracy_score(y_test, y_pred_xgb_lp)
    report_xgb_lp = classification_report(y_test, y_pred_xgb_lp, output_dict=True)

    # --- Cenário SEM LabelPropagation (descarta as amostras sem rótulo) ---
    # Selecionar apenas as amostras que permaneceram rotuladas
    mask_labeled = y_train_semi != -1
    X_train_no_lp = X_train[mask_labeled]
    y_train_no_lp = y_train_semi[mask_labeled]

    # Treinar Logistic Regression sem LabelPropagation
    modelo_log_no_lp = LogisticRegression(max_iter=1000)
    modelo_log_no_lp.fit(X_train_no_lp, y_train_no_lp)
    y_pred_log_no_lp = modelo_log_no_lp.predict(X_test)
    acc_log_no_lp = accuracy_score(y_test, y_pred_log_no_lp)
    report_log_no_lp = classification_report(y_test, y_pred_log_no_lp, output_dict=True)

    # Treinar XGBoost sem LabelPropagation
    modelo_xgb_no_lp = XGBClassifier(eval_metric='logloss')
    modelo_xgb_no_lp.fit(X_train_no_lp, y_train_no_lp)
    y_pred_xgb_no_lp = modelo_xgb_no_lp.predict(X_test)
    acc_xgb_no_lp = accuracy_score(y_test, y_pred_xgb_no_lp)
    report_xgb_no_lp = classification_report(y_test, y_pred_xgb_no_lp, output_dict=True)

    # Armazenar resultados desta proporção
    resultados.append({
        'proporcao_remocao': prop,
        # Com LabelPropagation
        'acc_log_lp': acc_log_lp,
        'precision_log_lp': report_log_lp['weighted avg']['precision'],
        'recall_log_lp': report_log_lp['weighted avg']['recall'],
        'f1_log_lp': report_log_lp['weighted avg']['f1-score'],
        'acc_xgb_lp': acc_xgb_lp,
        'precision_xgb_lp': report_xgb_lp['weighted avg']['precision'],
        'recall_xgb_lp': report_xgb_lp['weighted avg']['recall'],
        'f1_xgb_lp': report_xgb_lp['weighted avg']['f1-score'],
        # Sem LabelPropagation
        'acc_log_no_lp': acc_log_no_lp,
        'precision_log_no_lp': report_log_no_lp['weighted avg']['precision'],
        'recall_log_no_lp': report_log_no_lp['weighted avg']['recall'],
        'f1_log_no_lp': report_log_no_lp['weighted avg']['f1-score'],
        'acc_xgb_no_lp': acc_xgb_no_lp,
        'precision_xgb_no_lp': report_xgb_no_lp['weighted avg']['precision'],
        'recall_xgb_no_lp': report_xgb_no_lp['weighted avg']['recall'],
        'f1_xgb_no_lp': report_xgb_no_lp['weighted avg']['f1-score']
    })

# Converter resultados para DataFrame
df_resultados = pd.DataFrame(resultados)

# Salvar os resultados finais em CSV
df_resultados.to_csv('comparacao_labelpropagation_vs_sem.csv', index=False)
print("Resultados salvos em 'comparacao_labelpropagation_vs_sem.csv'.")


In [31]:
X_train_no_lp

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 139054 stored elements and shape (576, 79538)>